In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, mean_squared_error
import numpy as np
import joblib

In [2]:
data = pd.read_parquet('./input/creditos_hist.parquet')

In [3]:
# Eliminamos las situaciones 0, que indican que el crédito ya fue pagado
data = data.loc[data['situacion'] != 0]
data = data.drop('denominacion', axis = 1) # Elimino la columna con las razones sociales para ahorrar RAM

In [4]:
# Una variable que puede ser de interés es cuantos créditos tiene una empresa en un momento dado del tiempo
counts = data.groupby(['identificacion', 'periodo']).size().reset_index(name='n_creditos')

# También nos interesa cuanta plata debe una empresa en cada momento dado
sums = data.groupby(['identificacion', 'periodo'], as_index=True)['monto'].sum().reset_index(name='sum_montos')

# La literatura indica que también importa la duración de la relación empresa-banco, por lo que contamos la cantidad 
# de periodos que aparece cada par: empresa-banco
period_counts = data.groupby(['identificacion', 'entidad']).size().reset_index(name='n_periodos')

# Definimos como default cuando el crédito se encuentra en situación 4 o 5, por lo que creamos la dummy de default
# Esta es nuestra variable dependiente
data['default'] = (data['situacion'] >= 4).astype(int)

In [5]:
# Queremos predecir el default el periodo siguienre
data['default_lag'] = data.groupby(['identificacion', 'entidad'])['default'].shift(1) # Lag a la variable default
data = data.dropna(subset=['default_lag']) # Eliminamos las observaciones que no tienen variable dependiente
data['default_lag'] = data['default_lag'].astype(int) # Cambio el dtype de la variable de interés

In [6]:
data = data.sort_values(by=['identificacion', 'periodo'])
data['prev_default'] = (
    data.groupby('identificacion')['default']
    .transform(lambda x: x.cumsum().clip(upper=1))
) # Armamos una variable que indique si en algún momento de su historia, esa empresa tuvo un crédito en default

In [7]:
# Agregamos las nuevas variables al dataframe
data = data.merge(counts, on=['identificacion', 'periodo'], how='left')
data = data.merge(sums, on=['identificacion', 'periodo'], how='left')
data = data.merge(period_counts, on=['identificacion', 'entidad'], how='left')

del sums, counts, period_counts # Para ahorrar RAM

In [8]:
data = data.loc[data['periodo'] > '202310']

In [9]:
# Elijo aleatoriamente un porcentaje de las empresas de la población
np.random.seed(42)
cuits = data['identificacion'].unique()
moneda = np.random.binomial(1, 0.1, len(cuits)) # Es como tirar una moneda sesgada para que agarre un porcentaje arbitrario de las empresas 
cuits_aleatorios = cuits[moneda == 1] # Estos son los cuits con los que me voy a quedar
data = data.loc[data['identificacion'].isin(cuits_aleatorios)] # Me quedo unicamente con las obs que tienen un cuit dentro de los seleccionados aleatoriamente

In [10]:
pv = pd.read_parquet('./input/principales_variables.parquet') # Datos de principales variables monetarias provenientes de la API del BCRA
pv.reset_index(inplace= True) # el index es la fecha, así que lo paso a columna
pv['fecha'] = pd.to_datetime(pv['fecha']) # paso la nueva columna al formato correcto
pv['periodo'] = pv['fecha'].dt.strftime('%Y%m') # armo una variable llamada periodo igual a la que tengo en los datos de la Central de Deudores
pv = pv.drop('fecha', axis = 1).groupby('periodo').agg(['mean', 'std']) # elimino la de "fecha" porque no me interesan los datos diarios
# Me quedo únicamente con los promedios por mes y también calculo el desvío estándar
pv.columns = ['_'.join(col).strip() for col in pv.columns] # Renombro las columnas para que sea más prolijo
pv.reset_index(inplace= True) # Vuelvo a agregar la columna periodo
pv = pv.loc[pv['periodo'].astype(int) <= 202411] # En la Central de Deudores tenemos datos hasta 202410
pv = pv.dropna(axis = 1) # Elimino las columnas con NAs

In [11]:
data = data.merge(pv, on = 'periodo', how = 'left') # Junto las principales variables monetarias con la Central de Deudores

In [12]:
emae = pd.read_excel('./input/sh_emae_mensual_base2004.xls', index_col=[0,1])
meses_a_numeros = {
    'Enero': '01', 'Febrero': '02', 'Marzo': '03', 'Abril': '04',
    'Mayo': '05', 'Junio': '06', 'Julio': '07', 'Agosto': '08',
    'Septiembre': '09', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'
}
emae = emae.reset_index()
emae['level_1'] = emae['level_1'].map(meses_a_numeros)
emae['periodo'] = emae['Período'].astype(str) + emae['level_1']
emae = emae.drop(columns=['level_1', 'Período'])

In [13]:
data = data.merge(emae, on = 'periodo', how= 'left')

In [14]:
arca = pd.read_parquet('./input/constancia_inscripcion.parquet') # Cargo los datos de la constancia de inscripción de ARCA
#arca = arca.drop(['direccion', 'localidad', 'razonSocial'], axis = 1) # Elimino algunas variables para ahorrar RAM

In [15]:
cuits_arca = set(arca['identificacion'])
cuits_bcra = set(data['identificacion'])

faltan = list(cuits_bcra - cuits_arca)

data['sin_arca'] = (data['identificacion'].isin(faltan)).astype(int)

In [16]:
#data = data.merge(arca, on = 'identificacion', how= 'left') # Junto las bases

In [17]:
# Por último, la literatura también resalta que la intensidad de la relación empresa-banco es relevante
# Usamos como proxy para la intensidad la proporción del monto adeudado con un banco sobre el total adeudado
data['monto_relativo'] = data['monto'] / data['sum_montos']

In [18]:
# Ponemos bien el tipo de dato para las columnas categóricas, así el get_dummies funciona bien
data['identificacion'] = data['identificacion'].astype('category')
data['entidad'] = data['entidad'].astype('category')
data['situacion'] = data['situacion'].astype('category')
data['default'] = data['default'].astype('category')
data['periodo'] = data['periodo'].astype('category')
#data['codPostal'] = data['codPostal'].astype('category')
#data['mesCierre'] = data['mesCierre'].astype('category')
#data['provincia'] = data['provincia'].astype('category')
data['default_lag'] = data['default_lag'].astype('category')
data['prev_default'] = data['prev_default'].astype('category')
data['sin_arca'] = data['sin_arca'].astype('category')

In [19]:
#Y = data.loc[data['IVA'].notna()]['default_lag'] # Nuestra variable de interés para las empresas para las cuales tenemos datos de ARCA

Para hacer cross validation, tenemos que tener en cuenta que tenemos un panel. Por lo tanto, vamos a entrenar el modelo con datos del pasado y evaluarlo con datos del futuro

In [20]:
# Cross Validation
data = data.sort_values(by='periodo') # Ordeno de acuerdo a la fecha
data = data.reset_index().drop(columns= 'index')
split_index = int(len(data) * 0.8) # El 80% de las observaciones más antiguas
train_indices = data.iloc[:split_index].index # Estos son los índices con los que después voy a separar en test y train
test_indices = data.iloc[split_index:].index

In [21]:
Y = data['default_lag']

In [22]:
boolean_columns = data.select_dtypes(include='object').columns # Estas son las columnas que ya están en el formato correcto
columnas = ['entidad', 'monto', 'n_creditos', 'sum_montos', 'n_periodos', 'monto_relativo', 'sin_arca', 'default', 'prev_default'] # Algunas de las variables independientes del modelo
#columnas.extend(arca.columns) # Todas las columnas de ARCA
pv.set_index('periodo', inplace= True)
columnas.extend(pv.columns)# Todas las columnas de las principales variables monetarias
emae.set_index('periodo', inplace= True)
columnas.extend(emae.columns) # Todas las columnas de emae

In [23]:
columns_to_encode = [col for col in columnas if col not in boolean_columns] # Una lista con las columnas que no tengo que meter en "get_dummies"
X_encoded = pd.get_dummies(data[columns_to_encode], drop_first=True) # Meto las columnas en get_dummies
X = pd.concat([X_encoded, data[boolean_columns]], axis=1) # Junto todas las variables independientes en un solo df

del columns_to_encode, X_encoded

In [24]:
#X = data[data['IVA'].notna()] # Las X para las que si tenemos datos de ARCA

In [25]:
# Intersección entre conjunto de entrenamiento y tienen datos para ARCA
#train_indices = [idx for idx in train_indices if idx in X.index]
#test_indices = [idx for idx in test_indices if idx in X.index]

In [26]:
# Separo en entrenamiento y test
X_train = X.loc[train_indices]
Y_train = Y.loc[train_indices]
X_test = X.loc[test_indices]
Y_test = Y.loc[test_indices]

También se puede hacer con neg_mean_squared_error, accuracy, f1_macro, f1_samples, average_precision, etc

In [27]:
def eval(model, X_test, Y_test, linear = None):
    y_pred = model.predict(X_test)
    
    if linear:
        y_pred = np.where(y_pred >= 0.5, 1, 0)
    
    cm = confusion_matrix(Y_test, y_pred)
    
    precision = precision_score(Y_test, y_pred)
    recall = recall_score(Y_test, y_pred)
    f1 = f1_score(Y_test, y_pred)
    accuracy = accuracy_score(Y_test, y_pred)
    mse = mean_squared_error(Y_test, y_pred)
    
    print(cm)
    print(f'La precisión es: {precision}')
    print(f'El recall es: {recall}')
    print(f'El f1 es: {f1}')
    print(f'El accuracy es: {accuracy}')
    print(f'El MSE es: {mse}')
    
    return y_pred 
    

In [28]:
def mse_table(model, elasticnet=None):
    inverse_Cs = 1 / model.Cs_
    results = []
    
    if elasticnet:
        l1_ratios = model.l1_ratios_
        for c_idx, lambda_ in enumerate(inverse_Cs):
            for l1_idx, l1_ratio in enumerate(l1_ratios):
                    mean_score = np.mean(-model.scores_[1][:, c_idx, l1_idx], axis=0)
                    results.append({
                        "Lambda": lambda_,
                        "L1 Ratio": l1_ratio,
                        "Mean MSE": mean_score
                    })
        
        results_table = pd.DataFrame(results).sort_values(by="Mean MSE", ascending=True)
    
    else:
        mean_scores = np.mean(-model.scores_[1], axis=0)
        results_table = pd.DataFrame({
            "Lambda": inverse_Cs,
            "Mean Score": mean_scores
        }).sort_values(by="Mean Score", ascending=True)
    
    print(results_table)
    return results_table


In [29]:
def non_zero_coefs(model, X_train):
    best_coefs = model.coef_[0]
    feature_names = X_train.columns
    non_zero_coefs = []
    for coef, name in zip(best_coefs, feature_names):
        if coef != 0:
            non_zero_coefs.append({
                "Variable": name,
                "Coeficiente": coef
            })
    non_zero_table = pd.DataFrame(non_zero_coefs)
    print(non_zero_table)

## LASSO

In [28]:
tscv = TimeSeriesSplit(n_splits=10) # Este cross validation tiene en cuenta la temporarlidad de la base

pipeline = Pipeline([
    ('scaler', StandardScaler()), # Primero estandariza los datos
    ('logreg', LogisticRegressionCV( # Estima el modelo usando cross validation para elegir el mejor hiperparámetro
        cv=tscv,
        penalty='l1',
        solver='saga',
        scoring='neg_mean_squared_error',
        max_iter=2000,
        random_state= 42,
        tol = 1e-3,
        n_jobs= -1,
        fit_intercept= True,
        Cs=np.logspace(-4, 4, 20)
    ))
])

In [29]:
pipeline.fit(X_train, Y_train) # Entreno el modelo

Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegressionCV(Cs=array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                                      cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
                                      max_iter=2000, n_jobs=-1, penalty='l1',
                                      random_state=42,
                                      scoring='neg_mean_squared_error',
                                      solver='saga', tol=0.001))])

In [50]:
joblib.dump(pipeline, './output/lasso_completo_01.pkl')

['./output/lasso_completo_01.pkl']

In [35]:
pipeline = joblib.load('./output/lasso_completo_01.pkl')

In [104]:
y_pred = eval(pipeline, X_test, Y_test)

[[74142    36]
 [  258  4910]]
La precisión es: 0.9927213910230489
El recall es: 0.9500773993808049
El f1 es: 0.9709313822424362
El accuracy es: 0.9962947092481033
El MSE es: 0.0037052907518967558


In [31]:
model = pipeline.named_steps['logreg'] # Agarro el modelo desde el pipeline

In [32]:
best_c = model.C_
print(f'El mejor lambda para el modelo es: {1/best_c}')

El mejor lambda para el modelo es: [1438.44988829]


In [84]:
mse = mse_table(model)

          Lambda  Mean Score
2    1438.449888    0.018245
3     545.559478    0.018245
4     206.913808    0.018245
5      78.475997    0.018259
6      29.763514    0.018314
16      0.001833    0.018411
15      0.004833    0.018411
14      0.012743    0.018411
9       1.623777    0.018425
17      0.000695    0.018425
13      0.033598    0.018425
12      0.088587    0.018425
19      0.000100    0.018425
10      0.615848    0.018425
18      0.000264    0.018425
8       4.281332    0.018425
7      11.288379    0.018425
11      0.233572    0.018425
1    3792.690191    0.049355
0   10000.000000    0.069222


In [49]:
non_zero_coefs(model)

         Variable  Coeficiente
0      sin_arca_1     0.274379
1       default_1     1.642455
2  prev_default_1     0.358073


# Resultados Comparativos por Especificación

| Especificación       | **TN (PN)** | **FN (PN)** | **FP (PP)** | **TP (PP)** | **Accuracy (%)** |
|---------------------|-------------|-------------|-------------|-------------|------------------|
| **Base**            | 37,226      | 15          | 2,304       | 356         | 94.17           |
| **Base + prev_default** | 36,542  | 699         | 297         | 2,363       | 96.53           |
| **Base + default**  | 37,222      | 19          | 133         | 2,527       | 98.18           |
| **Base + sin_arca** | 37,099      | 142         | 1,692       | 968         | 95.05           |
| **Completo**        | 37,222      | 19          | 133         | 2,527       | 98.18           |


## Elastic Net

In [51]:
tscv = TimeSeriesSplit(n_splits=5)

pipeline_en = Pipeline([
    ('scaler', StandardScaler()),  
    ('logreg', LogisticRegressionCV(
        cv=tscv,
        penalty='elasticnet',
        solver='saga',
        scoring='neg_mean_squared_error',
        max_iter=2000,
        random_state=42,
        tol=1e-3,
        n_jobs=-1,
        fit_intercept=True,
        Cs=np.logspace(-4, 4, 10), 
        l1_ratios=np.linspace(0.1, 0.9, 9)  
    ))
])

In [52]:
pipeline_en.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegressionCV(Cs=array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04]),
                                      cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                                      l1_ratios=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                      max_iter=2000, n_jobs=-1,
                                      penalty='elasticnet', random_state=42,
                                      scoring='neg_mean_squared_error',
                                      solver='saga', tol=0.001))])

In [55]:
joblib.dump(pipeline_en, './output/elasticnet_completo_01.pkl')

['./output/elasticnet_completo_01.pkl']

In [56]:
pipeline_en = joblib.load('./output/elasticnet_completo_01.pkl')

In [105]:
y_pred = eval(pipeline_en, X_test, Y_test)

[[74142    36]
 [  258  4910]]
La precisión es: 0.9927213910230489
El recall es: 0.9500773993808049
El f1 es: 0.9709313822424362
El accuracy es: 0.9962947092481033
El MSE es: 0.0037052907518967558


In [58]:
elasticnet = pipeline_en.named_steps['logreg']
print(f'La proporción óptima de LASSO es: {elasticnet.l1_ratio_[0]}')

La proporción óptima de LASSO es: 0.30000000000000004


In [59]:
best_c = elasticnet.C_
print(f'El mejor lambda para el modelo es: {1/best_c}')

El mejor lambda para el modelo es: [10000.]


In [82]:
mse = mse_table(elasticnet, True)

          Lambda  L1 Ratio  Mean MSE
11   1291.549665       0.3  0.018285
2   10000.000000       0.3  0.018285
3   10000.000000       0.4  0.018285
17   1291.549665       0.9  0.018285
16   1291.549665       0.8  0.018285
..           ...       ...       ...
45      0.359381       0.1  0.090472
54      0.046416       0.1  0.090547
63      0.005995       0.1  0.090608
72      0.000774       0.1  0.090865
81      0.000100       0.1  0.090941

[90 rows x 3 columns]


In [100]:
non_zero_coefs(elasticnet, X_train)

         Variable  Coeficiente
0      sin_arca_1     0.229188
1       default_1     0.957084
2  prev_default_1     0.535358


## RandomForest

In [31]:
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(
        n_estimators=100, 
        max_depth=None,   
        random_state=42,  
        n_jobs=-1,
        oob_score = True,          
    ))
])

In [32]:
param_grid = {
    'rf__n_estimators': [100, 200, 400],
    'rf__max_depth': [None, 10, 20, 30],  
}

tscv = TimeSeriesSplit(n_splits=10)

grid_search = GridSearchCV(
    pipeline_rf, 
    param_grid, 
    cv=tscv, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1
)

In [33]:
grid_search.fit(X_train, Y_train)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rf',
                                        RandomForestClassifier(n_jobs=-1,
                                                               oob_score=True,
                                                               random_state=42))]),
             n_jobs=-1,
             param_grid={'rf__max_depth': [None, 10, 20, 30],
                         'rf__n_estimators': [100, 200, 400]},
             scoring='neg_mean_squared_error')

In [36]:
joblib.dump(grid_search, './output/rf_completo_01.pkl')

['./output/rf_completo_01.pkl']

In [ ]:
grid_search = joblib.load('./output/rf_completo_01.pkl')

In [34]:
y_pred = eval(grid_search, X_test, Y_test)

[[74142    36]
 [  258  4910]]
La precisión es: 0.9927213910230489
El recall es: 0.9500773993808049
El f1 es: 0.9709313822424362
El accuracy es: 0.9962947092481033
El MSE es: 0.0037052907518967558


In [40]:
rf = grid_search.best_estimator_.named_steps['rf']

In [46]:
importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf.feature_importances_
}).sort_values(by='Importance', ascending= False)
print(importances)

                                               Feature  Importance
251                                          default_1    0.478981
252                                     prev_default_1    0.292437
250                                         sin_arca_1    0.097717
2                                           sum_montos    0.023054
0                                                monto    0.017986
..                                                 ...         ...
154  entidad_Cooperativa Interamericana de Credito,...    0.000000
87   entidad_Asociación Mutual de Pensionados Socia...    0.000000
162                 entidad_DLL LEASING ARGENTINA S.A.    0.000000
83   entidad_ASOCIACION MUTUAL DE ASOCIADOS Y ADHER...    0.000000
178                              entidad_FULLCREDIT SA    0.000000

[253 rows x 2 columns]


## Modelo de Probabilidad Lineal

In [50]:
lm = LinearRegression(fit_intercept= True, n_jobs= -1)
lm.fit(X_train, Y_train)

LinearRegression(n_jobs=-1)

In [51]:
joblib.dump(lm, './output/lm_01.pkl')

['./output/lm_01.pkl']

In [52]:
lm = joblib.load('./output/lm_01.pkl')

In [53]:
y_pred = eval(lm, X_test, Y_test, linear= True)

[[10792 63386]
 [   40  5128]]
La precisión es: 0.07484601687246402
El recall es: 0.9922600619195047
El f1 es: 0.13919274721098776
El accuracy es: 0.2006402339122325
El MSE es: 0.7993597660877675


In [63]:
df_coeficientes = pd.DataFrame({
    'variable': X_train.columns,
    'coeficiente': lm.coef_,
    'abs_coef': np.abs(lm.coef_)
}).sort_values(by = 'abs_coef', ascending= False)
print(df_coeficientes.iloc[:,:2])

                                              variable   coeficiente
21   Depósitos en efectivo en las entidades financi... -1.758253e+03
27   En cuentas corrientes (neto de utilización FUC...  1.652047e+03
25      En Caja de ahorros (en millones de pesos)_mean  1.405382e+03
5    A plazo (incluye inversiones y excluye CEDROS)...  1.347137e+03
6    A plazo (incluye inversiones y excluye CEDROS)...  1.315129e+03
..                                                 ...           ...
87   entidad_Asociación Mutual de Pensionados Socia...  2.750394e-07
154  entidad_Cooperativa Interamericana de Credito,... -6.919741e-09
2                                           sum_montos -1.805411e-11
0                                                monto -3.961419e-14
205                               entidad_MIGXION S.A.  0.000000e+00

[253 rows x 2 columns]


## Regresión Logística

In [28]:
logit = LogisticRegression(max_iter= 2000, fit_intercept= True, n_jobs = -1, solver = 'saga')
logit.fit(X_train, Y_train)

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=2000, n_jobs=-1, solver='saga')

In [30]:
joblib.dump(logit, './output/logit_01.pkl')

['./output/logit_01.pkl']

In [55]:
logit = joblib.load('./output/logit_01.pkl')

In [56]:
y_pred = eval(logit, X_test, Y_test)

[[74178     0]
 [ 5168     0]]
La precisión es: 0.0
El recall es: 0.0
El f1 es: 0.0
El accuracy es: 0.9348675421571345
El MSE es: 0.06513245784286542


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [64]:
df_coeficientes = pd.DataFrame({
    'variable': X_train.columns,
    'coeficiente': logit.coef_[0],
    'abs_coef': np.abs(logit.coef_[0])
}).sort_values(by = 'coeficiente', ascending= False)
print(df_coeficientes.iloc[:, :2])

                                             variable   coeficiente
19  Depósitos de los bancos en cta. cte. en pesos ...  2.735048e-07
25     En Caja de ahorros (en millones de pesos)_mean  1.401358e-07
22  Depósitos en efectivo en las entidades financi...  1.234440e-07
5   A plazo (incluye inversiones y excluye CEDROS)...  1.200787e-07
6   A plazo (incluye inversiones y excluye CEDROS)...  1.111148e-07
..                                                ...           ...
36  Préstamos de las entidades financieras al sect... -2.391043e-07
27  En cuentas corrientes (neto de utilización FUC... -2.768733e-07
13  Billetes y monedas en poder del público (en mi... -2.854945e-07
17  Circulación monetaria (en millones de pesos)_mean -3.323784e-07
0                                               monto -7.917863e-07

[253 rows x 2 columns]
